# Sequence model
A sequence model is an older type of model that exposes raw sentances to train on. 



In [2]:
from tensorflow.keras import layers

In [3]:
max_length = 600
max_tokens = 20000
text_vec = layers.TextVectorization(max_tokens=max_tokens, output_mode = "int", output_sequence_length=max_length)

In [4]:
from tensorflow import keras
train_ds = keras.utils.text_dataset_from_directory("D:/Documents/Code/AI_training/training_data/aclImdb_v1/aclImdb/train", batch_size=32)
val_ds = keras.utils.text_dataset_from_directory("D:/Documents/Code/AI_training/training_data/aclImdb_v1/aclImdb/val",batch_size=32)
test_ds = keras.utils.text_dataset_from_directory("D:/Documents/Code/AI_training/training_data/aclImdb_v1/aclImdb/test",batch_size=32)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [5]:
text_only_train_ds = train_ds.map(lambda x,y:x)
text_vec.adapt(text_only_train_ds)

int_train_ds = train_ds.map(lambda x,y: (text_vec(x),y), num_parallel_calls=4)
int_val_ds = val_ds.map(lambda x,y: (text_vec(x),y), num_parallel_calls=4)
int_test_ds = test_ds.map(lambda x,y: (text_vec(x),y), num_parallel_calls=4)

In [6]:
import tensorflow as tf

inputs = keras.Input(shape=(None,), dtype="int64")
embedded = tf.one_hot(inputs, depth=max_tokens)
x = l


NameError: name 'l' is not defined

In [12]:
callbacks = keras.callbacks.ModelCheckpoint("one_hot_bidir_lstm.keras", save_best_only=True)
model.fit(int_train_ds.cache(), validation_data=int_val_ds.cache(), epochs=10, callbacks=callbacks)
model = keras.models.load_model("one_hot_bidir_lstm.keras")

print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")


Epoch 1/10
  2/625 [..............................] - ETA: 6:31:50 - loss: 0.6921 - accuracy: 0.5781

KeyboardInterrupt: 

Okay this is very very slow without a GPU! Unfortuntely Windows being the awful operating system that it is does not support graphical acceleration...

In [13]:
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))

Device mapping: no known devices.


# Embeddings
Up untiul now I have been using one-hot encoding which converts a word into a vector. However these vectors have no knowledge of other vacabularly. This is almost as if they have their own seperate orthogonal space. Emdings allow us to map the connections between words. 

In [ ]:
embedding_layer = layers.Embedding(input_dim=max_tokens, output_dim=256) # 256 is the dimensionality

In [9]:
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = layers.Embedding(input_dim=max_tokens, output_dim=256)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_2 (Embedding)     (None, None, 256)         5120000   
                                                                 
 bidirectional_2 (Bidirectio  (None, 64)               73984     
 nal)                                                            
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 5,194,049
Trainable params: 5,194,049
Non-trainable params: 0
_________________________________________________

In [10]:
callbacks = keras.callbacks.ModelCheckpoint("embeddings_bidir_gru.keras", save_best_only=True)
model.fit(int_train_ds.cache(), validation_data=int_val_ds.cache(), epochs=10, callbacks=callbacks)
model = keras.models.load_model("embeddings_bidir_gru.keras")

print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Epoch 1/10
625/625 [==============================] - 333s 524ms/step - loss: 0.4605 - accuracy: 0.7952 - val_loss: 0.3369 - val_accuracy: 0.8702
Epoch 2/10
625/625 [==============================] - 322s 516ms/step - loss: 0.3012 - accuracy: 0.8892 - val_loss: 0.3159 - val_accuracy: 0.8774
Epoch 3/10
625/625 [==============================] - 318s 509ms/step - loss: 0.2339 - accuracy: 0.9175 - val_loss: 0.3278 - val_accuracy: 0.8746
Epoch 4/10
625/625 [==============================] - 308s 493ms/step - loss: 0.1976 - accuracy: 0.9332 - val_loss: 0.3593 - val_accuracy: 0.8752
Epoch 5/10
625/625 [==============================] - 318s 509ms/step - loss: 0.1717 - accuracy: 0.9420 - val_loss: 0.3611 - val_accuracy: 0.8596
Epoch 6/10
625/625 [==============================] - 319s 510ms/step - loss: 0.1444 - accuracy: 0.9512 - val_loss: 0.3646 - val_accuracy: 0.8800
Epoch 7/10
625/625 [==============================] - 317s 508ms/step - loss: 0.1172 - accuracy: 0.9615 - val_loss: 0.4289 -

The above code truncates the sequence after 600 words so this is not yet as accurate as the bigrams method. A masking would work better to achieve slighty more accuracy. 

In [ ]:
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = layers.Embedding(input_dim=max_tokens, output_dim=256, mask_zero=True)(inputs) ### IMPORTANT ###
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()